In [1]:
import os
import pandas as pd

In [4]:
# 結合したいCSVファイルが存在するディレクトリを指定
#directory = './automation_med_best_result/'

# 結合したファイルを保存するためのファイル名
#output_file = 'automation_median_combined_reviews.csv'

# "re"で始まり"review.csv"で終わるファイルをリストに格納
#csv_files = [f for f in os.listdir(directory) if f.startswith('re') and f.endswith('review.csv')]

# 空のデータフレームを作成
#combined_df = pd.DataFrame()

# ファイルを順に読み込み、データフレームに結合
#for file in csv_files:
#    file_path = os.path.join(directory, file)
#    df = pd.read_csv(file_path)
#    combined_df = pd.concat([combined_df, df], ignore_index=True)

# 'トータル所要時間'が675を超えるデータを削除
#combined_df = combined_df[combined_df['トータル所要時間'] <= 675]

# 結合結果をCSVファイルとして保存
#combined_df.to_csv(os.path.join(directory, output_file), index=False)

#print(f"結合されたCSVファイルは {output_file} に保存されました。")


In [6]:
import os
import pandas as pd

# 複数のフォルダパスをリストで指定
directories = [
    '../automation_best_result_csv/',
    '../automation_med_best_result/',
    '../best_result_csv/',
    # 必要に応じてフォルダパスを追加
]

# 出力先のファイル名を指定
output_file = 'combined_data2.csv'

# 空のデータフレームを作成
combined_df = pd.DataFrame()

# 指定されたすべてのフォルダを走査
for directory in directories:
    # サブディレクトリも含めてCSVファイルを検索
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                df = pd.read_csv(file_path)
                
                # '日付'列が空欄の場合に'median'を入力
                df['日付'].fillna('median', inplace=True)
                
                # 'lambda'列を削除
                if 'lambda' in df.columns:
                    df.drop(columns=['lambda'], inplace=True)
                
                # 'トータル所要時間'が675以下のデータのみを抽出
                df_filtered = df[df['トータル所要時間'] <= 675]
                
                # フィルタリングされたデータを結合
                combined_df = pd.concat([combined_df, df_filtered], ignore_index=True)

display(combined_df.head())
# データをソート（例：'日付'列でソート）
combined_df.sort_values(by='日付', ascending=True, inplace=True)

# 結合したデータを1つのCSVファイルとして保存
combined_df.to_csv(output_file, index=False)

print(f"ソートされたCSVファイルは {output_file} に保存されました。")


,日付,トータル満足度,トータル所要時間,予定プラン,最適プラン,タイムスケジュール
0,2023-07-12,6074.0,662.0,"[5, 6, 4, 2, 7, 5, 4, 3, 4, 6, 5, 5, 6, 3]","[5, 6, 4, 2, 7, 5, 4, 3, 4, 6, 5, 6, 3]","[0, 30, 60, 120, 195, 255, 300, 345, 375, 435,..."
1,2023-07-12,6114.0,652.0,"[4, 7, 7, 4, 5, 2, 6, 3, 6, 6, 4, 5, 5, 4, 4, ...","[4, 7, 4, 5, 2, 6, 3, 6, 4, 5, 4, 1, 4]","[0, 30, 90, 150, 195, 270, 330, 345, 405, 465,..."
2,2023-07-12,6094.0,668.0,"[4, 6, 7, 4, 3, 3, 6, 3, 5, 4, 1, 5, 6, 4, 1, ...","[4, 6, 7, 4, 3, 6, 3, 5, 4, 1, 5, 6, 4]","[0, 30, 75, 135, 195, 270, 330, 345, 390, 465,..."
3,2023-07-12,6133.0,647.0,"[4, 5, 2, 4, 2, 5, 6, 3, 4, 4, 7, 4, 3, 4]","[4, 5, 2, 4, 2, 5, 6, 3, 4, 7, 4, 3, 4]","[0, 30, 60, 135, 195, 270, 300, 360, 375, 435,..."
4,2023-07-12,6117.0,670.0,"[2, 7, 3, 5, 4, 4, 4, 3, 4, 5, 2, 5, 4, 2, 4]","[2, 7, 3, 5, 4, 3, 4, 5, 2, 5, 4, 2, 4]","[0, 60, 120, 210, 255, 315, 330, 390, 435, 495..."


ソートされたCSVファイルは combined_data2.csv に保存されました。


In [7]:
import pandas as pd
import ast

# popularityリスト（各アトラクションの人気度）
popularity = [476, 465, 473, 472, 480, 452, 461, 478]

# CSVファイルを読み込み
df = pd.read_csv('combined_data2.csv')

# '最適プラン'の総和を計算し、'トータル満足度'に反映
for index, row in df.iterrows():
    # '最適プラン'をリストに変換（文字列をリストに変換）
    try:
        optimal_plan = ast.literal_eval(row['最適プラン'])
    except (ValueError, SyntaxError):
        print(f"エラー: 行 {index} の '最適プラン' を解析できません")
        continue
    
    # 人気度の総和を計算
    total_popularity = sum(popularity[i] for i in optimal_plan)
    
    # トータル満足度に反映
    df.at[index, 'トータル満足度'] = total_popularity

# 結果を確認
#print(df)

# 必要に応じてCSVに保存
df.to_csv('updated_combined_data2.csv', index=False)
